##### ПИ18-1 Гриднев Д.В.

In [1]:
import time
import numpy as np
from scipy.stats import sem
from sklearn import metrics, svm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer

Достаём обучающую и тестовую выборки, оставив только текст

In [2]:
train = fetch_20newsgroups(subset="train",remove=("headers", "footers", "quotes"))
test = fetch_20newsgroups(subset="test", remove=("headers", "footers", "quotes"))
all_set = fetch_20newsgroups(subset="all", remove=("headers", "footers", "quotes"))

In [3]:
train = fetch_20newsgroups(subset="train")
test = fetch_20newsgroups(subset="test")
all_set = fetch_20newsgroups(subset="all")

In [4]:
clf = Pipeline([
    ('vect', TfidfVectorizer(token_pattern=r"\b[a-z0-9_\-\.]+[a-z][a-z0-9_\-\.]+\b",)),
    ('clf', MultinomialNB(alpha=0.01)),
])

In [5]:
clf.fit(train.data, train.target)


Pipeline(steps=[('vect',
                 TfidfVectorizer(token_pattern='\\b[a-z0-9_\\-\\.]+[a-z][a-z0-9_\\-\\.]+\\b')),
                ('clf', MultinomialNB(alpha=0.01))])

In [6]:
print(train.data[2])

From: twillis@ec.ecn.purdue.edu (Thomas E Willis)
Subject: PB questions...
Organization: Purdue University Engineering Computer Network
Distribution: usa
Lines: 36

well folks, my mac plus finally gave up the ghost this weekend after
starting life as a 512k way back in 1985.  sooo, i'm in the market for a
new machine a bit sooner than i intended to be...

i'm looking into picking up a powerbook 160 or maybe 180 and have a bunch
of questions that (hopefully) somebody can answer:

* does anybody know any dirt on when the next round of powerbook
introductions are expected?  i'd heard the 185c was supposed to make an
appearence "this summer" but haven't heard anymore on it - and since i
don't have access to macleak, i was wondering if anybody out there had
more info...

* has anybody heard rumors about price drops to the powerbook line like the
ones the duo's just went through recently?

* what's the impression of the display on the 180?  i could probably swing
a 180 if i got the 80Mb disk

Метрики

In [7]:
predict = clf.predict(test.data)
print(f"""
accuracy: {metrics.accuracy_score(test.target, predict)}
precision: {metrics.precision_score(test.target, predict, average='weighted', zero_division=0)}
recall: {metrics.recall_score(test.target, predict, average='weighted', zero_division=0)}
""")



accuracy: 0.8462559745087627
precision: 0.8478256434323562
recall: 0.8462559745087627



In [8]:
cv = KFold(5, shuffle=True, random_state=0).get_n_splits(len(train.target))
scores = cross_val_score(clf, all_set.data, all_set.target, cv=cv)
print(f"Mean: {np.mean(scores)} ±{sem(scores)}")

Mean: 0.9219461712293434 ±0.0018819152501145934


Другие классификаторы

In [ ]:
models = [
    LogisticRegression(max_iter=10000, n_jobs=-1),
    svm.SVC(kernel='linear'),
    svm.SVC(kernel='rbf'),
    KNeighborsClassifier(n_jobs=-1),
    MLPClassifier(max_iter=10000)
]

for model in models:
    start_time = time.time()
    clf = Pipeline([
    ('vect', TfidfVectorizer(
                token_pattern=r"\b[a-z0-9_\-\.]+[a-z][a-z0-9_\-\.]+\b",
    )),
    ('clf', model),
])
    clf.fit(train.data, train.target)

    spend_time = time.time() - start_time
    predict = clf.predict(test.data)
    print(
        clf,
        "\nЗатрачено секунд ",
        spend_time,
        "\nScore",
        clf.score(test.data, test.target),
        "\nМетрика достоверности предсказания",
        metrics.accuracy_score(test.target, predict),
        "\nМетрика точности",
        metrics.precision_score(test.target, predict, average="weighted", zero_division=0),
        "\nМетрика полноты",
        metrics.recall_score(test.target, predict,average="weighted", zero_division=0),
        "\n"
    )

Pipeline(steps=[('vect',
                 TfidfVectorizer(token_pattern='\\b[a-z0-9_\\-\\.]+[a-z][a-z0-9_\\-\\.]+\\b')),
                ('clf', LogisticRegression(max_iter=10000, n_jobs=-1))]) 
Затрачено секунд  47.688963890075684 
Score 0.830987785448752 
Метрика достоверности предсказания 0.830987785448752 
Метрика точности 0.8353616648830646 
Метрика полноты 0.830987785448752 

Pipeline(steps=[('vect',
                 TfidfVectorizer(token_pattern='\\b[a-z0-9_\\-\\.]+[a-z][a-z0-9_\\-\\.]+\\b')),
                ('clf', SVC(kernel='linear'))]) 
Затрачено секунд  134.57499980926514 
Score 0.8353690918746681 
Метрика достоверности предсказания 0.8353690918746681 
Метрика точности 0.8404139136691945 
Метрика полноты 0.8353690918746681 

Pipeline(steps=[('vect',
                 TfidfVectorizer(token_pattern='\\b[a-z0-9_\\-\\.]+[a-z][a-z0-9_\\-\\.]+\\b')),
                ('clf', SVC())]) 
Затрачено секунд  350.6934621334076 
Score 0.8208975039830059 
Метрика достоверности предсказания

Лучше всего LogisticRegression, он же оказался самым быстрым

KNeighbors не смог обучиться

MLPClassifier хоть и оказался самым точным, но работал больше 45 минут. Там конечно стоит ставить ограничение на кол-во веток меньше :)